# First make sure all the files are ready for analysis

In [1]:
import sys
sys.path.append('/home/itskov/Dropbox/workspace/lab/DeepSemantic/WormSegmentation')

import matplotlib
matplotlib.use('TkAgg')

from glob2 import glob
from Behavior.General.ExpDir import ExpDir
from Behavior.General.Experiment import Experiment
from os import listdir, path
import pandas as pd


# Debug (Temporary)
from Behavior.General import Track
sys.modules['Track'] = Track
# Debug (Temporary)

import numpy as np
import re

print('Starting..')

# Here is where the tracked files are stored.
#rootDir = '/mnt/storageNASRe/tph1/Results/24-Nov-2019'
#rootDir = '/mnt/storageNASRe/ChristianData/ChrisNewTracks/'
rootDir = '/home/itskov/Temp/behav/05-Dec-2019/'
allTrackedFiles = glob(rootDir + '/**/*tracks.npy')
allTrackedDirectories = [path.dirname(tracksFile) for tracksFile in allTrackedFiles]


# Here we store our experiments meta information
expDf = pd.DataFrame(columns=['date','time', 'cond','fullFile'])

for i, trackedDir in enumerate(allTrackedDirectories):
    fullFile = path.join(rootDir, trackedDir)
    print("%d. Going over %s" % (i, fullFile))
    
    # This object helps us manage the tracked files.
    expDir = ExpDir(str(fullFile))
    
    #if True:
    if (expDir.getExpFile() == None) or (not expDir.isValid()):
        #newExp = Experiment(expDir.getVidFile(), np.load(expDir.getTracksFile()))
        newExp = Experiment(expDir)
        newExp.takeScale()
        newExp.addCirclePotisionRad('startReg',newExp._scale / 5)
        newExp.addCirclePotisionRad('endReg',newExp._scale / 4)
        newExp.save()
        pass
        
        
    currentDate = re.search('\d\d-\w\w\w-\d\d\d\d',trackedDir)[0]
    currentTime = re.search('_(\d\d\.\d\d\.\d\d)',path.basename(fullFile))[1]
    cond = re.search('(.*)\.avi',path.basename(fullFile))[1]
    
    # Irregularities
    #if cond == 'STAP_Mock':
    #    cond = 'MOCK_STAP'
    #elif cond == 'LTAV_MOCK':
    #    cond = 'MOCK_LTAV'
    #elif cond == 'TEMP_EYAL_LIGHT':
    #    continue
    
    expDf.loc[i] = [currentDate, currentTime, cond, fullFile]
    #print('Date: ' + currentDate + ' Cond: ' + cond)
expDf

Starting..
0. Going over /home/itskov/Temp/behav/05-Dec-2019/MOCK_STAP.avi_12.57.19
1. Going over /home/itskov/Temp/behav/05-Dec-2019/STAP.avi_12.13.59
{}
Initializing Experiment with dir: /home/itskov/Temp/behav/05-Dec-2019/STAP.avi_12.13.59 and with 25486 tracks
{}
2. Going over /home/itskov/Temp/behav/05-Dec-2019/MOCK_STAP.avi_12.13.37
{}
Initializing Experiment with dir: /home/itskov/Temp/behav/05-Dec-2019/MOCK_STAP.avi_12.13.37 and with 35757 tracks
{}
3. Going over /home/itskov/Temp/behav/05-Dec-2019/STAP.avi_12.58.39
{}
Initializing Experiment with dir: /home/itskov/Temp/behav/05-Dec-2019/STAP.avi_12.58.39 and with 46030 tracks
{}


,date,time,cond,fullFile
0,05-Dec-2019,12.57.19,MOCK_STAP,/home/itskov/Temp/behav/05-Dec-2019/MOCK_STAP....
1,05-Dec-2019,12.13.59,STAP,/home/itskov/Temp/behav/05-Dec-2019/STAP.avi_1...
2,05-Dec-2019,12.13.37,MOCK_STAP,/home/itskov/Temp/behav/05-Dec-2019/MOCK_STAP....
3,05-Dec-2019,12.58.39,STAP,/home/itskov/Temp/behav/05-Dec-2019/STAP.avi_1...



# Pair analyses

In [14]:
import sys
sys.path.append('/home/itskov/workspace/lab/DeepSemantic/WormSegmentation')


import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, widgets, VBox,HBox

from Behavior.Visualizers.PairwiseAnalyses import PairWiseRoi
from Behavior.Visualizers.PairwiseAnalyses import PairWiseProjectionDensity, PairWiseOccupVisoulatizer, PairWiseSpeedDensity, PairWiseRevDistances
from Behavior.General.TracksFilter import filterTracksForAnalyses



# First experiment widgets
#print(pd.unique(expDf['date']))
dateWidget1 = widgets.Dropdown(options=pd.unique(expDf['date']),value=expDf['date'][0], description='Date')
condWidget1 = widgets.Dropdown(options=pd.unique(expDf['cond']),value=expDf['cond'][0], description='Cond')
timeWidget1 = widgets.Dropdown(options=[], description='Time')

# Second experiment widgets
dateWidget2 = widgets.Dropdown(options=pd.unique(expDf['date']),value=expDf['date'][0], description='Date')
condWidget2 = widgets.Dropdown(options=pd.unique(expDf['cond']),value=expDf['cond'][0], description='Cond')
timeWidget2 = widgets.Dropdown(options=[], description='Time')

out = widgets.Output()
display(out)

@out.capture()
def runAnalysis(arg):
    firstCond = condWidget1.value
    firstDate = dateWidget1.value
    firstTime = timeWidget1.value
    secondCond = condWidget2.value
    secondDate = dateWidget2.value
    secondTime = timeWidget2.value
    
    firstFile = expDf[(expDf.cond == firstCond) & 
                      (expDf.date == firstDate) & 
                      (expDf.time == firstTime)]['fullFile']
    
    secondFile = expDf[(expDf.cond == secondCond) & 
                       (expDf.date == secondDate) & 
                       (expDf.time == secondTime)]['fullFile']
    
    firstFile = path.join(firstFile.values[0],'exp.npy')
    secondFile = path.join(secondFile.values[0],'exp.npy')
    
    print(firstFile)
    print(secondFile)
    firstExp = np.load(firstFile)[0]
    secondExp = np.load(secondFile)[0]
    
    # Shortenning the experiments.
    firstExp.trimExperiment(4500)
    secondExp.trimExperiment(4500)
        
    print('Filtering tracks for anaylses..')
    #firstExp._tracks = filterTracksForAnalyses(firstExp._tracks)
    #secondExp._tracks = filterTracksForAnalyses(secondExp._tracks)

    PairWiseRoi(firstCond, firstExp, secondCond, secondExp)
    PairWiseProjectionDensity(firstCond, firstExp, secondCond, secondExp)
    PairWiseSpeedDensity(firstCond, firstExp, secondCond, secondExp)
    PairWiseOccupVisoulatizer(firstCond, firstExp, secondCond, secondExp)
    #PairWiseRevDistances(firstCond, firstExp, secondCond, secondExp)
    

# Run button
runButton = widgets.Button(description="Run")
runButton.on_click(runAnalysis)

def on_cond_value_change1(*change):        
    dateWidget1.options = pd.unique(expDf[expDf['cond'] == condWidget1.value]['date'])
    timeWidget1.options = pd.unique(expDf[expDf['cond'] == condWidget1.value]['time'])
    
def on_date_value_change1(*change):
    timeWidget1.options = expDf[(expDf['cond'] == condWidget1.value) & (expDf['date'] == dateWidget1.value) ]['time']
    
def on_cond_value_change2(*change):        
    dateWidget2.options = pd.unique(expDf[expDf['cond'] == condWidget2.value]['date'])
    timeWidget2.options = pd.unique(expDf[expDf['cond'] == condWidget2.value]['time'])

def on_date_value_change2(*change):
    timeWidget2.options = expDf[(expDf['cond'] == condWidget2.value) & (expDf['date'] == dateWidget2.value) ]['time']

condWidget1.observe(on_cond_value_change1, names='value')
dateWidget1.observe(on_date_value_change1, names='value')
condWidget2.observe(on_cond_value_change2, names='value')
dateWidget2.observe(on_date_value_change2, names='value')

# Display layout
HBox([VBox([condWidget1, dateWidget1, timeWidget1]), VBox([condWidget2, dateWidget2, timeWidget2]), runButton])

Output()

# Create Pair Movie


In [2]:
import sys
sys.path.append('/home/itskov/workspace/lab/DeepSemantic/WormSegmentation')


from Behavior.General.ExpPair import ExpPair

expPair = ExpPair('/home/itskov/Temp/behav/05-Dec-2019/STAP.avi_12.13.59/exp.npy', 
                  '/home/itskov/Temp/behav/05-Dec-2019/MOCK_STAP.avi_12.13.37/exp.npy')

expPair._firstExp.trimExperiment(4501)
expPair._secondExp.trimExperiment(4501)

expPair.createPairVisualization(4500, dpi=250)

Reinitializing.
Reinitializing.
Trimming Experiment to 3653 frames.
Trimming Experiment to 3709 frames.
Allocating space.
Filtering tracks. Before 25486, After: 15123
Allocating space.
Filtering tracks. Before 35757, After: 22672
Processed Frame 3649 / 3652

AxisError: axis 1 is out of bounds for array of dimension 0

# Create ROI Artifact

In [ ]:
import sys
sys.path.append('/home/itskov/workspace/lab/DeepSemantic/WormSegmentation')


from pathlib import Path
from os import path
from Behavior.Tools.Artifacts import Artifacts
from Behavior.Visualiz

import numpy as np

rootDir = '/mnt/storageNASRe/tph1/Results/19-Nov-2019/'

for fileName in Path(rootDir).rglob('exp.npy'):
    print(fileName)
            
    # load experiment.
    exp = np.load(fileName)[0]
    
    # Create an artifact folder.
    art = Artifacts(exp)
    
    exp
    
    
        
    

# Re-Initialize Experiemnt in a new Location

In [5]:
import sys
sys.path.append('/home/itskov/workspace/lab/DeepSemantic/WormSegmentation')

import numpy as np

from os import path
from Behavior.General.ExpDir import ExpDir
from glob import glob

expFile = '/home/itskov/Temp/behav/28-Nov-2019/TPH_1_NO_ATR_TRAIN_NO_IAA3x5.avi_17.13.10/exp.npy'
currentExp = np.load(expFile)[0]
expDir = currentExp.initialize(ExpDir(path.dirname(expFile)))
print('Saving..')
currentExp.save()


'''DIR = '/mnt/storageNASRe/tph1/Results/28-Nov-2019/'

globPath = DIR + '**/exp.npy'
print(globPath)
expFiles = glob(globPath)

for i,expFile in enumerate(expFiles):
    print("Gover over %s" %  (expFile,))
    currentExp = np.load(expFile)[0]
    expDir = currentExp.initialize(ExpDir(path.dirname(expFile)))
    print('Saving..')
    currentExp.save()
    print('Done.')
    '''
    
    

Reinitializing.
{'startReg': {'pos': (1056.1233766233763, 2095.0844155844156), 'rad': 231.40626942259428}, 'endReg': {'pos': (952.2272727272725, 941.8376623376622), 'rad': 289.25783677824285}}
Initializing Experiment with dir: /home/itskov/Temp/behav/28-Nov-2019/TPH_1_NO_ATR_TRAIN_NO_IAA3x5.avi_17.13.10 and with 37743 tracks
{'startReg': {'pos': (1056.1233766233763, 2095.0844155844156), 'rad': 231.40626942259428}, 'endReg': {'pos': (952.2272727272725, 941.8376623376622), 'rad': 289.25783677824285}}
Saving..


'DIR = \'/mnt/storageNASRe/tph1/Results/28-Nov-2019/\'\n\nglobPath = DIR + \'**/exp.npy\'\nprint(globPath)\nexpFiles = glob(globPath)\n\nfor i,expFile in enumerate(expFiles):\n    print("Gover over %s" %  (expFile,))\n    currentExp = np.load(expFile)[0]\n    expDir = currentExp.initialize(ExpDir(path.dirname(expFile)))\n    print(\'Saving..\')\n    currentExp.save()\n    print(\'Done.\')\n    '